In [1]:
import pandas as pd
import os

In [2]:
pwd

'C:\\E Drive\\Mixed Materials\\AI-ML\\Python Code and Practice\\KaggleData\\netflix-prize-data'

In [2]:
wd = 'C:\\E Drive\\Mixed Materials\\AI-ML\\Python Code and Practice\\KaggleData\\netflix-prize-data\\'

In [3]:
data = [wd + 'combined_data_{}.txt' .format(i) for i in range (1,5)]

In [4]:
data

['C:\\E Drive\\Mixed Materials\\AI-ML\\Python Code and Practice\\KaggleData\\netflix-prize-data\\combined_data_1.txt',
 'C:\\E Drive\\Mixed Materials\\AI-ML\\Python Code and Practice\\KaggleData\\netflix-prize-data\\combined_data_2.txt',
 'C:\\E Drive\\Mixed Materials\\AI-ML\\Python Code and Practice\\KaggleData\\netflix-prize-data\\combined_data_3.txt',
 'C:\\E Drive\\Mixed Materials\\AI-ML\\Python Code and Practice\\KaggleData\\netflix-prize-data\\combined_data_4.txt']

In [5]:
get_movie_id = lambda line: int(line.replace(':', '').replace('\n', ''))
get_rating = lambda line: [x.replace('\n', '') for x in line.split(',')]
df_it = lambda row: pd.DataFrame(row, index=['cust_id', 'rating', 'date']).T

def append_to_csv(data, fp):
    print('Writing {} rows to {}'.format(data.shape[0], fp))
    if os.path.exists(fp):
        data.to_csv(fp, mode='a', index=False, header=False)
    else:
        data.to_csv(fp, index=False)

In [1]:
def get_ratings(fp):
    """
    Parse the text files that have movie id and customer ratings into a usable dataframe.
    @fp: file path (str)
    """
    print('Getting ratings from file: {}'.format(fp))
    agg_data = []
    with open(fp, 'r') as file_:
        for line_number, line in enumerate(file_):
#             if (line_number % 10**7) == 0: print('{} million rows..'.format(line_number / (10**6)))
            if ':' in line:
                movie_id = get_movie_id(line)                      
            else:
                rating_row = get_rating(line)
                rating = {str(col): val for col,val in enumerate(rating_row)}
                rating['movie_id'] = movie_id
                agg_data.append(rating)            
    agg_data = pd.DataFrame(agg_data)
    agg_data.rename(columns={'0': 'cust_id', '1': 'rating', '2': 'date'}, inplace=True)
    print('Finished getting ratings from file: {}'.format(fp))
    return agg_data
#for fp in data_files:
df = get_ratings(data[0])
append_to_csv(df, 'ratings.csv')

In [30]:
df.head(10)

,,13368:
2385003,4.0,2004-07-08
659432,3.0,2005-03-16
751812,2.0,2002-12-16
2625420,2.0,2004-05-25
1650301,1.0,2005-08-30
2269227,4.0,2005-10-27
2220672,4.0,2002-08-19
13369:,NaN,NaN
1625310,5.0,2005-03-17
541295,5.0,2005-04-08


In [3]:
df2 = pd.read_csv(r'movie_titles.csv',encoding = "ISO-8859-1",names = ['movieId','YearOfRelease','Title'])

In [4]:
df3 = pd.read_csv('ratings.csv')

In [5]:
df2.head(10)

,movieId,YearOfRelease,Title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


In [12]:
df3.head(3)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523


In [6]:
df_final = df2.merge(df3,on = 'movieId',how = 'left')

In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21105171 entries, 0 to 21105170
Data columns (total 6 columns):
movieId          int64
YearOfRelease    float64
Title            object
userId           float64
rating           float64
timestamp        float64
dtypes: float64(4), int64(1), object(1)
memory usage: 1.1+ GB


In [8]:
df_final.isna().sum()

movieId             0
YearOfRelease    5291
Title               0
userId           9433
rating           9433
timestamp        9433
dtype: int64

In [9]:
df_final['userId'].value_counts().head(10)

8659.0      6161
179792.0    5730
107720.0    5169
45811.0     4363
229879.0    4097
70648.0     3791
172224.0    3746
270123.0    3634
59554.0     3595
243443.0    3557
Name: userId, dtype: int64

In [10]:
df_final.dropna(inplace=True)

In [11]:
df_final.head(10)

,movieId,YearOfRelease,Title,userId,rating,timestamp
0,1,2003.0,Dinosaur Planet,8.0,4.0,1.013444e+09
1,1,2003.0,Dinosaur Planet,9.0,4.5,1.073837e+09
2,1,2003.0,Dinosaur Planet,12.0,4.0,9.439122e+08
3,1,2003.0,Dinosaur Planet,20.0,4.0,1.368361e+09
4,1,2003.0,Dinosaur Planet,24.0,4.0,9.798699e+08
5,1,2003.0,Dinosaur Planet,27.0,3.5,1.201213e+09
6,1,2003.0,Dinosaur Planet,34.0,3.0,1.498761e+09
7,1,2003.0,Dinosaur Planet,37.0,3.5,1.301635e+09
8,1,2003.0,Dinosaur Planet,40.0,4.0,8.625155e+08
9,1,2003.0,Dinosaur Planet,41.0,3.5,1.445256e+09


In [12]:
from surprise import Dataset,Reader

In [13]:
reader =  Reader(rating_scale=(1, 5))

In [14]:
data = Dataset.load_from_df(df_final[['userId','Title','rating']], reader)

In [15]:
from surprise.model_selection import train_test_split

In [16]:
trainset,testset = train_test_split(data, test_size=0.25, random_state=123)

In [17]:
from surprise import KNNWithMeans
from surprise import accuracy

In [18]:
knn_model = KNNWithMeans(k = 10, sim_options= {'name': 'pearson', 'user_based': False} )

In [19]:
knn_model.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [20]:
knn_model.get_neighbors(1000, k = 2)

[211, 296]

In [21]:
test_pred = knn_model.test(testset)

In [22]:
accuracy.rmse(test_pred)

RMSE: 0.8349


0.8348987613633078

In [23]:
test_pred[10]

Prediction(uid=88175.0, iid="Ocean's Eleven", r_ui=3.5, est=3.421897246593698, details={'actual_k': 10, 'was_impossible': False})

In [24]:
test_pred_df = pd.DataFrame(test_pred)
test_pred_df['was_impossible'] = [x['was_impossible'] for x in test_pred_df['details']]

In [42]:
# df_final
df_final[(df_final['userId'] == 63955) & (df_final['Title'] == 'Destination Moon')]

,movieId,YearOfRelease,Title,userId,rating,timestamp
7879549,1240,1950.0,Destination Moon,63955.0,0.5,1.457589e+09


In [34]:
test_pred_df[test_pred_df.was_impossible == False]

,uid,iid,r_ui,est,details,was_impossible
0,63955.0,Destination Moon,0.5,1.000000,"{'actual_k': 10, 'was_impossible': False}",False
1,190400.0,Calcutta Mail,2.0,2.259114,"{'actual_k': 10, 'was_impossible': False}",False
2,173647.0,The Importance of Being Earnest,4.5,3.860665,"{'actual_k': 10, 'was_impossible': False}",False
3,245451.0,El Grito,4.0,2.732002,"{'actual_k': 10, 'was_impossible': False}",False
4,151811.0,The Prophecy 3: The Ascent,4.5,3.357765,"{'actual_k': 10, 'was_impossible': False}",False
5,78384.0,Wings of Desire,3.0,3.243713,"{'actual_k': 10, 'was_impossible': False}",False
6,72909.0,Bawarchi,4.0,4.230215,"{'actual_k': 10, 'was_impossible': False}",False
7,236882.0,The Roaring Twenties,3.0,4.083142,"{'actual_k': 10, 'was_impossible': False}",False
8,84957.0,Chicago with Earth,3.5,3.055040,"{'actual_k': 10, 'was_impossible': False}",False
9,56368.0,Backyardigans: It's Great to Be a Ghost,4.0,3.555041,"{'actual_k': 10, 'was_impossible': False}",False
